# Installation

In [ ]:
# !pip install -r requirements.txt
# !pip install -q tqdm
# !pip install transformers==4.49.0
# !pip install python-dotenv
# !pip install google-api-python-client==2.100.0 gspread==5.10.0
# !pip install huggingface-hub==0.29.1
# !pip install sqlglot
# !pip install langchain-community

## Download model into local

In [ ]:
## Download model into local
import getpass
from huggingface_hub import hf_hub_download
from huggingface_hub import login
HF_TOKEN = getpass.getpass("Enter your Hugging Face token: ")
login(token=HF_TOKEN)

/home/myenv-text-to-sql/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import os

# Set the model repository name (change this to the desired model)
model_repo = "mistralai/Mistral-7B-Instruct-v0.2"
# branch = "refs/pr/1"
branch = "main"
# Define the directory where the model will be saved
save_directory = os.path.join("text_to_sql/models", model_repo)

# Create the directory if it does not exist
os.makedirs(save_directory, exist_ok=True)

# Use the Hugging Face CLI to download the model
download_cmd = f"huggingface-cli download {model_repo} --revision {branch} --local-dir {save_directory}"
# os.system(download_cmd)
# prompt = "huggingface-cli download Ellbendls/Qwen-2.5-3b-Text_to_SQL --local-dir models/Ellbendls/Qwen-2.5-3b-Text_to_SQL"
# print(f"Model downloaded to {save_directory}")
download_cmd

'huggingface-cli download mistralai/Mistral-7B-Instruct-v0.2 --revision main --local-dir text_to_sql/models/mistralai/Mistral-7B-Instruct-v0.2'

# Inference Model

## Load data from GDrive

In [14]:
import os
import pandas as pd
import json

from typing import List
from dotenv import load_dotenv

load_dotenv()

GOOGLE_SPREADSHEET_ID: str = "1dDMqrol_DrEMjvLy88IRu2WdHN7T5BU0LrD8ORLuNPI" # put your spreadsheet id here
GOOGLE_SPREADSHEET_URL: str = f"https://docs.google.com/spreadsheets/d/{GOOGLE_SPREADSHEET_ID}/edit?usp=sharing" # put your spreadsheet link here
DATA_TEST_SHEET_NAME: str = "test_data"

GOOGLE_SHEETS_CLIENT_EMAIL: str = os.getenv('GOOGLE_SHEETS_CLIENT_EMAIL')
GOOGLE_SHEETS_PRIVATE_KEY: str = os.getenv('GOOGLE_SHEETS_PRIVATE_KEY')

In [5]:
## Google Auth
# Google Authentication
from modules.google_sheets_writer import GoogleUtil
from IPython.display import display, Markdown

PRIVATE_KEY = GOOGLE_SHEETS_PRIVATE_KEY
google: GoogleUtil = GoogleUtil(PRIVATE_KEY, GOOGLE_SHEETS_CLIENT_EMAIL)


In [18]:
## Load Data Test
rows: List[list] = google.retrieve_worksheet(GOOGLE_SPREADSHEET_ID, DATA_TEST_SHEET_NAME)
df_data_test: pd.DataFrame = pd.DataFrame(rows[1:], columns=rows[0])
display(df_data_test.head(2))

,No,Id,Prompt,Expected SQL Query,Expected Query Result,Sheet,Database
0,1,1,Bagaimana perbandingan jumlah karyawan berdasa...,"SELECT organizations.name AS ""organization_nam...",[{'organization_name': 'Information Technology...,catapa_test_core_employee,core
1,2,2,Bagaimana data termination berdasarkan nama ja...,"SELECT job_titles.name AS ""job_title_name"", CO...","[{'job_title_name': 'Information Technology', ...",catapa_test_core_employee,core


## Inference Model

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_core.prompts import PromptTemplate
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from peft import PeftModel

import torch
model_dir = "unsloth"  # based on your base model path.
model_name = "Qwen2.5-Coder-7B-Instruct" # based on your base model path.
model_id = os.path.join(model_dir, model_name)
sft_path = "/home/text_to_sql/fine_tuned/fine_tune_exp_1/unsloth/Qwen2.5-Coder-7B-Instruct/exp_id_1:fine_tuning_ft-1:ft_text_to_sql_few_shot_1"
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16)
model = PeftModel.from_pretrained(base_model, sft_path)  # uncomment if you have fine tuned model
model = model.merge_and_unload()  # uncomment if you have fine tuned model

pipe = pipeline(
    task="text-generation",
    model=base_model,
    device=0,
    torch_dtype=torch.bfloat16,
    tokenizer=tokenizer,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    max_new_tokens=512,
    return_full_text=False,
    model_kwargs = {"temperature": 0, "do_sample":True},
)

hf_pipeline = HuggingFacePipeline(pipeline=pipe)

/home/text_to_sql_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.66it/s]
Device set to use cuda:0
/tmp/ipykernel_1830/3968238050.py:31: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  hf_pipeline = HuggingFacePipeline(pipeline=pipe)


In [9]:
SYSTEM_PROMPT = """You are a SQL generator expert for MariaDB 10.5.23. Your task is to create SQL queries based on database schema, table relationships, master data, current date, and user instructions.

1. CORE SQL REQUIREMENTS:
   - Generate directly executable MariaDB 10.5.23 SQL
   - DO NOT select identifiers (id, employee_id) except within aggregate functions (MAX, SUM, AVG)
   - Always use human-readable names (religions.name instead of religions.id)
   - Prefix all column names with table names to avoid ambiguity
   - Use snake_case for column aliases in SELECT clause only (no aliases in FROM)
   - Handle dates properly:
     * Wrap all date literals in STR_TO_DATE()
     * Ensure date comparisons use CAST() for DATE type
   - Use aggregate functions ONLY in SELECT or HAVING clauses (never in WHERE)
   - Ensure JOIN conditions reference correct foreign keys
   - Add extra JOINs to master tables if display names are missing
   - Handle division by checking for non-zero denominators
   - Name output columns based on user's instruction language

2. DATA TRUSTEE REQUIREMENTS:
   - If any table in the query appears in the Data Trustee Enabled Tables list:
     * JOIN with the employment_statuses table using employee_id column
     * Add these exact filters:
       employment_statuses.organization_id IN ('[ORGANIZATION_IDS]')
       AND employment_statuses.job_level_id IN ('[JOB_LEVEL_IDS]')
       AND employment_statuses.location_id IN ('[LOCATION_IDS]')
     * If table requires prerequisite join (indicated in data_trustee_tables),
       perform that JOIN before joining with employment_statuses
   - If no table from the Data Trustee Enabled Tables is used, do not include
     any data trustee-specific JOINs or filters

3. OUTPUT FORMAT:
   - Return ONLY a JSON object with your SQL query in this exact format:
        {{
        "sql_query": "<your_generated_sql_query>"
        }}
   - No explanations or commentary

Examples:
Input: "Berapa total uang yang ditransfer untuk karyawan yang aktif di bulan Oktober 2023?"

Output: {
  "sql_query": "SELECT SUM(salary_payment_summaries.transferred_amount) AS 'total_transferred_amount' FROM salary_payment_summaries JOIN salary_payments ON salary_payment_summaries.id = salary_payments.salary_payment_summary_id JOIN employees ON salary_payments.employee_id = employees.id JOIN employment_statuses ON employees.id = employment_statuses.employee_id WHERE employees.active = TRUE AND employment_statuses.organization_id IN ([ORGANIZATION_IDS]) AND employment_statuses.job_level_id IN ([JOB_LEVEL_IDS]) AND employment_statuses.location_id IN ([LOCATION_IDS]) AND salary_payment_summaries.payment_date BETWEEN STR_TO_DATE('2023-10-01', '%Y-%m-%d') AND STR_TO_DATE('2023-10-31', '%Y-%m-%d');"
}
"""

USER_PROMPT = """Generate a SQL query for the following instruction:

DATABASE INFORMATION:
Schema: {schema}
Relationships: {relations}
Master Data: {master_data}
Data Trustee Enabled Tables: {data_trustee_tables}
Anonymized Entities: {anonymized_entities_description}
Current Date: {current_date}

user_instruction: {user_instruction}
Return only the SQL query as a JSON object: {{"sql_query": "<your_sql_query>"}}"""

In [22]:
from datetime import datetime
from time import time
from tqdm import tqdm
from modules.database_info.schema import employee_schema, time_management_schema
from modules.database_info.master_data import employee_master_data, time_management_master_data
from modules.database_info.relation import employee_relations, time_management_relations
from modules.database_info.trustee_tables import data_trustee_employee, data_trustee_time_management
from modules.database_info.anonymize_entities import anonymized_entities_description


results = []

inference_result_dir = "sql_generator_result"
os.makedirs(inference_result_dir, exist_ok=True)
current_date = datetime.now().strftime("%d %B %Y")
sql_generator_result_path = os.path.join(inference_result_dir, f"{model_name}-using-catapa-prompt.csv")


for index, df_row in tqdm(enumerate(df_data_test.iterrows()), desc="Generating SQL queries", total=len(df_data_test)):
    user_instruction = df_row[1]['Prompt']
    no = df_row[1]['No']
    if "Id" in df_row[1]:
        id = df_row[1]['Id']
    else:
        id = "-"
    database_type = df_row[1]['Database']
    if not os.path.exists(sql_generator_result_path):
        df_query_result = pd.DataFrame(
            columns=[
                'No',
                'Id',
                'Prompt',
                'Generated SQL Query',
                'Expected SQL Query',
                'Expected Query Result',
                'Database',
                'Time Taken'
            ]
        )
        df_query_result.to_csv(sql_generator_result_path, index=False)
    else:
        df_query_result = pd.read_csv(sql_generator_result_path)

    if int(no) in df_query_result['No'].to_list():
        print(f"Skipping {no} because it already exists")
        continue
    # if index > 0:  # 15, 25
    #     continue

    start_time = time()
    if database_type == "core":
        schema = employee_schema
        relations = employee_relations
        master_data = employee_master_data
        data_trustee_tables = data_trustee_employee
        master_data = employee_master_data
    else:
        schema = time_management_schema
        relations = time_management_relations
        master_data = time_management_master_data
        data_trustee_tables = data_trustee_time_management
        master_data = time_management_master_data

    formatted_user_prompt = USER_PROMPT.format(
        schema=schema,
        relations=relations,
        master_data=master_data,
        data_trustee_tables=data_trustee_tables,
        anonymized_entities_description=anonymized_entities_description,
        current_date=current_date,
        user_instruction=user_instruction,
        query_result = ""
    )
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": formatted_user_prompt},
    ]
    # Check if the tokenizer has a chat template
    has_chat_template = hasattr(tokenizer, "chat_template") and tokenizer.chat_template is not None

    if has_chat_template:
        # Tokenize input with chat template
        inputs = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    else:
        inputs = messages

    sql_generator_result = hf_pipeline.invoke(inputs)
    # print(sql_generator_result)
    time_taken = time() - start_time
    # Create DataFrame with explicit index to avoid ValueError
    df_query_generator = pd.DataFrame({
        'No': [no],  # Wrap scalar values in lists
        'Id': [id],
        'Prompt': [user_instruction],
        'Generated SQL Query': [sql_generator_result],
        'Expected SQL Query': [df_row[1]['Expected SQL Query']],
        'Expected Query Result': [df_row[1]['Expected Query Result']],
        'Database': [database_type],
        'Time Taken': [time_taken]
    })
    # Concatenate with existing results
    df_query_result = pd.concat([df_query_result, df_query_generator], ignore_index=True)
    df_query_result.to_csv(sql_generator_result_path, index=False)
display(df_query_result)

Generating SQL queries:   0%|          | 0/102 [00:00<?, ?it/s]

Skipping 1 because it already exists


Generating SQL queries:   2%|▏         | 2/102 [00:03<02:32,  1.53s/it]

In [ ]:
sql_generator_result_path

'sql_generator_result/CodeLlama-7b-Instruct-hf-using-catapa-prompt.csv'

## Upload to Google Sheets

In [ ]:
from modules.google_sheets_writer import GoogleSheetsWriter
import logging

INFERENCE_RESULT_WORKSHEET = "experiment_inference_result"

writer = GoogleSheetsWriter(
    google_util=google,  # Your GoogleUtil instance
    sheet_id=GOOGLE_SPREADSHEET_ID,
    worksheet_name="experiment_inference_result",
    batch_size=10,  # Customize batch size
    max_retries=5,  # Customize retry attempts
    batch_delay=2  # Customize delay between batches
)
# Write the DataFrame
result = writer.write_dataframe(df_query_result)

# Log results
logging.info(f"Successfully wrote {result.successful_rows} rows")
if result.failed_rows > 0:
    logging.error(f"Failed to write {result.failed_rows} rows")
    for error in result.errors:
        logging.error(f"Row {error['row_number']}: {error['error']}")

# =============================================================================